<a href="https://colab.research.google.com/github/code-sanya/UCLA_Technology_Studio/blob/main/StyleTransfer_Pytorch_Unity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UCLA.AUD.IDEAS TECHNOLOGY SEMINAR 
#**PYTORCH STYLETRANSFER NOTEBOOK FOR UNITY**

**IMPORTANT:** Read all text before running the cell as some directions can change parameters or might download to the incorrect folder 
Read the output of each cell after you run it > if you get an error or something didnt load or it cant find a folder or image then SOMETHING IS WRONG. 

#**PART 01 - NOTEBOOK ENVIRONMENT SETUP**

In [ ]:
#@title 01:  Install fastai library (built on PyTorch)
%%capture
!pip install fastai==2.2.5

In [ ]:
#@title 02:  Import Dependencies, Modules, Packages
# Miscellaneous operating system interfaces
# https://docs.python.org/3/library/os.html
import os
# Time access and conversions
# https://docs.python.org/3/library/time.html
import time
# Object-oriented filesystem paths
# https://docs.python.org/3/library/pathlib.html#pathlib.Path
from pathlib import Path
# Tuple-like objects that have named fields
# https://docs.python.org/3/library/collections.html#collections.namedtuple
from collections import namedtuple

# A convenience function for downloading files from a url to a destination folder
# https://docs.fast.ai/data.external.html#untar_data
from fastai.data.external import untar_data

# Provides image processing capabilities
# https://pillow.readthedocs.io/en/stable/reference/Image.html
from PIL import Image

# The main PyTorch package
# https://pytorch.org/docs/stable/torch.html
import torch

# Used to iterate over the dataset during training 
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
from torch.utils.data import DataLoader

# Contains definitions of models. We'll be downloading a pretrained VGG-19 model
# to judge the performance of our style transfer model.
# https://pytorch.org/vision/stable/models.html#torchvision.models.vgg19
from torchvision.models import vgg19
# Common image transforms that we'll use to process images before feeding them to the models
# https://pytorch.org/vision/stable/transforms.html
from torchvision import transforms
# Loads images from a directory and applies the specified transforms
# https://pytorch.org/vision/stable/datasets.html#imagefolder
from torchvision.datasets import ImageFolder

In [ ]:
#@title 03:  Define some Functions
def make_dir(dir_name: str):
    """Create the specified directory if it doesn't already exist"""
    dir_path = Path(dir_name)
    try:
        dir_path.mkdir()
    except:
        print("Directory already exists.")

def load_image(filename: str, size: int=None, scale: float=None):
    """Load the specified image and return it as a PIL Image"""
    img = Image.open(filename)
    if size is not None:
        img = img.resize((size, size), Image.ANTIALIAS)
    elif scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image.ANTIALIAS)
    return img

def save_image(filename: str, data: torch.Tensor):
    """Save the Tensor data to an image file"""
    img = data.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype("uint8")
    img = Image.fromarray(img)
    img.save(filename)

def load_checkpoint(model_path):
    state_dict = torch.load(model_path)
    keys = [k for k in state_dict.keys()]
    filters = set()
    filters_list = [state_dict[k].shape[0] for k in keys if not (state_dict[k].shape[0] in filters or filters.add(state_dict[k].shape[0]))]
    res_blocks = len(set(k.split('.')[1] for k in state_dict.keys() if 'resnets' in k))
    model = TransformerNet(filters=filters_list[:-1], res_blocks=res_blocks) 
    model.load_state_dict(state_dict, strict=False)
    return model

def stylize(model_path: str, input_image: str, output_image: str, content_scale: float=None, 
            device: str="cpu", export_onnx: bool=None):
    """Load a TransformerNet checkpoint, stylize an image and save the output"""
    device = torch.device(device)
    content_image = load_image(input_image, scale=content_scale)
    content_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255))
    ])
    content_image = content_transform(content_image)
    content_image = content_image.unsqueeze(0).to(device)

    with torch.no_grad():
        style_model = load_checkpoint(model_path)
        style_model.to(device)
         
        if export_onnx:
            assert export_onnx.endswith(".onnx"), "Export model file should end with .onnx"
            output = torch.onnx._export(style_model, content_image, export_onnx, opset_version=9).cpu()
        else:
            output = style_model(content_image).cpu()
    save_image(output_image, output[0])

In [ ]:
#@title 04:  Define the Style Transfer Model
#@markdown Here we'll define the style transfer model itself. The model takes in an RGB image and generates a new image with the same dimensions. The features in the output image (e.g. color and texure) are then compared with the features of the style image and content image. The results of these comparisons are then used to update the parameters of the model so that it hopefully generates better images.

class TransformerNet(torch.nn.Module):
    """TransformerNet
    https://github.com/pytorch/examples/blob/36441a83b6595524a538e342594ee6482754f374/fast_neural_style/neural_style/transformer_net.py#L4
    """
    
    def __init__(self, filters=(32, 64, 128), res_blocks=5):
        super(TransformerNet, self).__init__()
        self.filters = filters
        self.res_blocks = res_blocks if res_blocks > 0 else 1
        # Initial convolution layers
        self.conv1 = ConvLayer(3, filters[0], kernel_size=9, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(filters[0], affine=True)
        self.conv2 = ConvLayer(filters[0], filters[1], kernel_size=3, stride=2)
        self.in2 = torch.nn.InstanceNorm2d(filters[1], affine=True)
        self.conv3 = ConvLayer(filters[1], filters[2], kernel_size=3, stride=2)
        self.in3 = torch.nn.InstanceNorm2d(filters[2], affine=True)
        # Residual layers
        self.resnets = torch.nn.ModuleList()
        for i in range(self.res_blocks):
            self.resnets.append(ResidualBlock(filters[2]))
        
        # Upsampling Layers
        self.deconv1 = UpsampleConvLayer(filters[2], filters[1], kernel_size=3, stride=1, upsample=2)
        self.in4 = torch.nn.InstanceNorm2d(filters[1], affine=True)
        self.deconv2 = UpsampleConvLayer(filters[1], filters[0], kernel_size=3, stride=1, upsample=2)
        self.in5 = torch.nn.InstanceNorm2d(filters[0], affine=True)
        self.deconv3 = ConvLayer(filters[0], 3, kernel_size=9, stride=1)
        # Non-linearities
        self.relu = torch.nn.ReLU()
        
    def forward(self, X):
        conv1_y = self.relu(self.in1(self.conv1(X)))
        conv2_y = self.relu(self.in2(self.conv2(conv1_y)))
        conv3_y = self.relu(self.in3(self.conv3(conv2_y)))

        y = self.resnets[0](conv3_y) + conv3_y
        
        for i in range(1, self.res_blocks):
            y = self.resnets[i](y) + y

        y = self.relu(self.in4(self.deconv1(conv3_y + y)))
        y = self.relu(self.in5(self.deconv2(conv2_y + y)))
        y = self.deconv3(conv1_y + y)
        return y


class ConvLayer(torch.nn.Module):
    """ConvLayer
    https://github.com/pytorch/examples/blob/36441a83b6595524a538e342594ee6482754f374/fast_neural_style/neural_style/transformer_net.py#L44
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out


class ResidualBlock(torch.nn.Module):
    """ResidualBlock
    introduced in: https://arxiv.org/abs/1512.03385
    recommended architecture: http://torch.ch/blog/2016/02/04/resnets.html
    https://github.com/pytorch/examples/blob/36441a83b6595524a538e342594ee6482754f374/fast_neural_style/neural_style/transformer_net.py#L57
    """

    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.relu = torch.nn.ReLU()
      
    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out


class UpsampleConvLayer(torch.nn.Module):
    """UpsampleConvLayer
    Upsamples the input and then does a convolution. This method gives better results
    compared to ConvTranspose2d.
    ref: http://distill.pub/2016/deconv-checkerboard/
    https://github.com/pytorch/examples/blob/36441a83b6595524a538e342594ee6482754f374/fast_neural_style/neural_style/transformer_net.py#L79
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        
    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out

In [ ]:
#@title 05:  Define the VGG-19 Model
#@markdown Next, we'll define the model that will be used to judge the quality of the output images from the style transfer model. This model has been pretrained a large image dataset. We'll use this model to extract the features of the content image, style image, and stylized images.
class Vgg19(torch.nn.Module):
    """
    https://github.com/pytorch/examples/blob/36441a83b6595524a538e342594ee6482754f374/fast_neural_style/neural_style/vgg.py#L7
    """
    
    def __init__(self, requires_grad=False):
        super(Vgg19, self).__init__()
        self.feature_layers = [0, 3, 5]
        self.vgg_pretrained_features = vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(4):
            self.slice1.add_module(str(x), self.vgg_pretrained_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x), self.vgg_pretrained_features[x])
        for x in range(9, 18):
            self.slice3.add_module(str(x), self.vgg_pretrained_features[x])
        for x in range(18, 27):
            self.slice4.add_module(str(x), self.vgg_pretrained_features[x])
        for x in range(27, 36):
            self.slice5.add_module(str(x), self.vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False
            
    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        h = self.slice5(h)
        h_relu5_3 = h
        vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3', 'relu5_3'])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3, h_relu5_3)
        return out

In [ ]:
#@title 06:  Define the Model Trainer
#@markdown This is still setting everything up
class Trainer(object):
    def __init__(self, train_loader, style_transform, generator, opt_generator, style_criterion, perception_model, device):
        self.train_loader = train_loader
        self.style_transform = style_transform
        self.generator = generator
        self.opt_generator = opt_generator
        self.style_criterion = style_criterion
        self.perception_model = perception_model
        self.device = device
        self.generator.to(self.device)
        
    def gram_matrix(self, y: torch.Tensor):
        """Compute the gram matrix a PyTorch Tensor"""
        (b, ch, h, w) = y.size()
        features = y.view(b, ch, w * h)
        features_t = features.transpose(1, 2)
        gram = features.bmm(features_t) / (ch * h * w)
        return gram

    def normalize_batch(self, batch: torch.Tensor):
        """Normalize a batch of Tensors using the imagenet mean and std """
        mean = batch.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
        std = batch.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
        batch = batch.div_(255.0)
        return (batch - mean) / std

    def get_gram_style(self, style_image: str, style_size: int):
        """Get the Gram Matrices for the style image"""
        style = load_image(style_image, size=style_size)
        style = self.style_transform(style)
        style = style.repeat(self.train_loader.batch_size, 1, 1, 1).to(self.device)
        features_style = self.perception_model(self.normalize_batch(style))
        gram_style = [self.gram_matrix(y) for y in features_style]
        return gram_style
            
    def save_checkpoint(self, path: str):
        """Save the current model weights at the specified path"""
        self.generator.eval().cpu()
        torch.save(self.generator.state_dict(), path)
        print(f"Checkpoint saved at {path}")

    def train(self, style_image, test_image, checkpoint_model_dir, epochs=5, content_weight=1e5, style_weight=1e10, 
                content_scale=None, style_size=None, log_interval=500, checkpoint_interval=500):
        """Train the style transfer model on the provided style image."""
        
        gram_style = self.get_gram_style(style_image, style_size)

        for e in range(epochs):
            self.generator.train()
            agg_content_loss = 0.
            agg_style_loss = 0.
            count = 0
            for batch_id, (x, _) in enumerate(self.train_loader):
                n_batch = len(x)
                count += n_batch
                self.opt_generator.zero_grad()
                
                x = x.to(self.device)
                y = self.generator(x)

                y = self.normalize_batch(y.clone())
                x = self.normalize_batch(x.clone())
                features_y = self.perception_model(y)
                features_x = self.perception_model(x)

                content_loss = content_weight * self.style_criterion(features_y.relu2_2, features_x.relu2_2)

                style_loss = 0.
                for ft_y, gm_s in zip(features_y, gram_style):
                    gm_y = self.gram_matrix(ft_y)
                    style_loss += self.style_criterion(gm_y, gm_s[:n_batch, :, :])
                style_loss = style_loss * style_weight

                total_loss = content_loss + style_loss
                total_loss.backward()
                self.opt_generator.step()

                agg_content_loss += content_loss.item()
                agg_style_loss += style_loss.item()

                if (batch_id + 1) % log_interval == 0:
                    mesg = f"{' '.join(time.ctime().replace('  ', ' ').split(' ')[1:-1])}  "
                    mesg += f"Epoch {e + 1}: [{count}/{len(self.train_loader.dataset)}]  "
                    mesg += f"content: {(agg_content_loss / (batch_id + 1)):.4f}  "
                    mesg += f"style: {(agg_style_loss / (batch_id + 1)):.4f}  "
                    mesg += f"total: {((agg_content_loss + agg_style_loss) / (batch_id + 1)):.4f}"
                    print(mesg)

                if checkpoint_model_dir is not None and (batch_id + 1) % checkpoint_interval == 0:
                    ckpt_base = f"ckpt_epoch_{e}_batch_id_{batch_id + 1}"
                    ckpt_model_filename = ckpt_base + ".pth"
                    ckpt_model_path = os.path.join(checkpoint_model_dir, ckpt_model_filename)
                    self.save_checkpoint(ckpt_model_path)
                    output_image = ckpt_base + ".png"
                    output_image_path = os.path.join(checkpoint_model_dir, output_image)
                    stylize(ckpt_model_path, test_image, output_image_path)
                    self.generator.to(self.device).train()
                
        print("Finished Training")
        ckpt_model_path = os.path.join(checkpoint_model_dir, 'final.pth')
        self.save_checkpoint(ckpt_model_path)
        output_image_path = os.path.join(checkpoint_model_dir, 'final.png')
        stylize(ckpt_model_path, test_image, output_image_path)

#**PART 02 - FOLDER SETUP**

In [ ]:
#@title 01:  Mount to your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title 02: Make a Folder in your drive named `styleTransfer-Pytorch`
#@markdown And sets some folder directories
%cd /content/gdrive/MyDrive
!mkdir styleTransfer-Pytorch
%cd /content/gdrive/MyDrive/styleTransfer-Pytorch
!mkdir style_images
!mkdir test_images
!mkdir dataset
!mkdir checkpoints
!mkdir ./dataset/video_frames/

dataset_dir = "/content/dataset"

project_dir = '/content/gdrive/MyDrive/styleTransfer-Pytorch'
style_images_dir = f"{project_dir}/style_images"
test_images_dir = f"{project_dir}/test_images"
checkpoints_dir = f"{project_dir}/checkpoints"
make_dir(checkpoints_dir)

!ls $project_dir

/content/gdrive/MyDrive
mkdir: cannot create directory ‘styleTransfer-Pytorch’: File exists
/content/gdrive/MyDrive/styleTransfer-Pytorch
mkdir: cannot create directory ‘style_images’: File exists
mkdir: cannot create directory ‘test_images’: File exists
mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘checkpoints’: File exists
mkdir: cannot create directory ‘./dataset/video_frames/’: File exists
Directory already exists.
Akira.onnx  checkpoints  Landscape.onnx  Ruins2.onnx  style_images
BNW.onnx    dataset	 Lightmode.onnx  Ruins.onnx   test_images


Now save test images and style images in their respective folders. Style images and test images need to be JPG and the dataset images can be either png or jpg

In [ ]:
#@title 03:  Download the COCO Training Dataset
#@markdown We'll be using the COCO train 2014 image dataset to train. It contains `82,783` images.     
#@markdown Note: It's about 13.5 GB unzipped. You might get a disk space warning while the dataset is being unzipped. You can click ignore in the popup window.  
coco_url = 'http://images.cocodataset.org/zips/train2014.zip'
untar_data(coco_url, 'coco.zip', dataset_dir)
if os.path.exists('coco.zip'): os.remove('coco.zip')

In [ ]:
#@title 04:  Set some parameters for the model
batch_size = 4
image_size = 256
transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Lambda(lambda x: x.mul(255))
                                ])

train_dataset = ImageFolder(dataset_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

use_cuda = True
device = "cuda" if (use_cuda and torch.cuda.is_available()) else "cpu"
print(f"Using: {device}")

style_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Lambda(lambda x: x.mul(255))
                                      ])

filters = (8, 16, 32)
res_blocks = 5
generator = TransformerNet(filters=filters, res_blocks=res_blocks).to(device)

lr = 1e-3
opt_generator = torch.optim.Adam(generator.parameters(), lr)

style_criterion = torch.nn.MSELoss()

Using: cuda


In [ ]:
#@title 05:  Create a New VGG-19 Perception Model
perception_model = Vgg19(requires_grad=False).to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

#**PART 03: TRAINING SECTION**

In [ ]:
#@title 01: Create a New Trainer 
trainer = Trainer(train_loader=train_loader, 
                  style_transform=style_transform, 
                  generator=generator, 
                  opt_generator=opt_generator, 
                  style_criterion=style_criterion, 
                  perception_model=perception_model, 
                  device=device)



In [ ]:
#@title 02: Set Training Parameters
#@markdown Make sure the style_image and test_image are both .jpg and 256x256       

#@markdown Change the style_image, epochs, and style_weight

# The file path for the target style image
style_image = f"{style_images_dir}/Soundmode.jpg"
# The file path for a sample input image for demonstrating the model's progress during training
test_image = f"{test_images_dir}/mesh.jpg" 

# The number of times to iterate through the entire training dataset
epochs = 6

# The influence from the input image on the stylized image - Default: 1e5
content_weight = 1e5 
# The influence from the style image on the stylized image - Default: 1e10
style_weight = 2e10

# (test_image resolution) / content_scale
content_scale = 2.0 #1
# Target size for style_image = (style_size, styl_size)
style_size = 256

# The number of training batches to wait before printing the progress of the model 
log_interval = 500
# The number of training to wait before saving the current model weights
checkpoint_interval = 500

In [ ]:
#@title 03: TRAIN
trainer.train(style_image=style_image, 
              test_image=test_image, 
              checkpoint_model_dir=checkpoints_dir, 
              epochs=epochs, 
              content_weight=content_weight, 
              style_weight=style_weight,
              content_scale=content_scale,
              style_size=style_size,
              log_interval=log_interval, 
              checkpoint_interval=checkpoint_interval)

Feb 5 03:11:00  Epoch 1: [2000/82783]  content: 531426.1534  style: 1148601.9171  total: 1680028.0706
Checkpoint saved at /content/gdrive/MyDrive/styleTransfer-Pytorch/checkpoints/ckpt_epoch_0_batch_id_500.pth
Feb 5 03:12:28  Epoch 1: [4000/82783]  content: 550411.9532  style: 783805.9152  total: 1334217.8684
Checkpoint saved at /content/gdrive/MyDrive/styleTransfer-Pytorch/checkpoints/ckpt_epoch_0_batch_id_1000.pth
Feb 5 03:13:55  Epoch 1: [6000/82783]  content: 553938.4809  style: 629814.4238  total: 1183752.9047
Checkpoint saved at /content/gdrive/MyDrive/styleTransfer-Pytorch/checkpoints/ckpt_epoch_0_batch_id_1500.pth
Feb 5 03:15:23  Epoch 1: [8000/82783]  content: 554121.8854  style: 543166.4145  total: 1097288.2999
Checkpoint saved at /content/gdrive/MyDrive/styleTransfer-Pytorch/checkpoints/ckpt_epoch_0_batch_id_2000.pth
Feb 5 03:16:50  Epoch 1: [10000/82783]  content: 551446.1323  style: 486480.4409  total: 1037926.5732
Checkpoint saved at /content/gdrive/MyDrive/styleTransfer-

#**PART 04: EXPORT FOR UNITY**

In [ ]:
#@title Export ONNX File >> Unity
#@markdown Make sure to change the checkpoint .pth name and the exported onnx file name to something that relates to the style so you remember.       
#@markdown The ONNX file will be saved to the project folder in Google Drive.
checkpoint_path = f"{checkpoints_dir}/ckpt_epoch_5_batch_id_20500.pth"
style_model = load_checkpoint(checkpoint_path)
x = torch.randn(1, 3, 960, 540).cpu()

torch.onnx.export(style_model.cpu(),     #  Model being run
                  x,                           # Sample input
                  f"{project_dir}/Soundmode.onnx", # Path to save ONNX file
                  export_params=True,          # Store trained weights
                  opset_version=9,             # Which ONNX version to use
                  do_constant_folding=True     # Replace operations that have all constant inputs with pre-computed nodes
                 )

/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:267: UserWarning: You are trying to export the model with onnx:Upsample for ONNX opset version 9. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator. 
  "" + str(_export_onnx_opset_version) + ". "


You can now drag this ONNX file into your StyleTransfer > models folder and apply it to the camera's style transfer script!

///

This notebook was put together by Laure Michelon for the WInter Quarter Tech Seminar @UCLA IDEAS

github - https://github.com/laure-m